# LIVRABLE 1 - Classification binaire (Projet LEYENDA)

# Objectif : Distinguer les photos naturelles (couleur ou noir et blanc) des autres types d'images (peintures, textes, dessins, schémas...)

CHANGER EN PNG, RESIZE
MODELE RGB
MODELE NB
RECONNAITRE PHOTO PEINTURE


In [ ]:
# =======================
# Partie 1 : Initialisation & Vérification GPU
# =======================

# Importations générales
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
# ✅ Vérification de la disponibilité GPU (TensorFlow GPU Metal)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("[INFO] GPU détecté :", gpus)
else:
    print("[WARNING] Pas de GPU détecté. Vérifie tensorflow-metal.")

In [ ]:
# =======================
# Partie 2 : Organisation initiale des dossiers
# =======================

# Chemins des dossiers
source_dir = "./images"
working_dir = "./images_work"

# Auto-détection des catégories présentes dans source_dir
categories = [d for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))]

# Création du dossier de travail et copie des images originales
os.makedirs(working_dir, exist_ok=True)

for category in categories:
    src_path = os.path.join(source_dir, category)
    dst_path = os.path.join(working_dir, category)

    if not os.path.exists(dst_path):
        shutil.copytree(src_path, dst_path)
        print(f"[COPIED] Catégorie : {category}")
    else:
        print(f"[ALREADY EXISTS] Catégorie : {category}")

In [ ]:
# =======================
# Partie 3 : Tri automatique en Couleur et Noir & Blanc
# =======================

# Dossier de travail et catégories
categories = ["Painting", "Photo", "Schematics", "Sketch", "Text"]

# Fonction pour détecter si une image est en niveau de gris
def is_grayscale(img_path):
    img = Image.open(img_path).convert("RGB")
    np_img = np.array(img)
    return np.all(np_img[:,:,0] == np_img[:,:,1]) and np.all(np_img[:,:,1] == np_img[:,:,2])

# Tri des images selon la couleur
for category in categories:
    path = os.path.join(working_dir, category)

    # Création des sous-dossiers rgb et nb
    rgb_path = os.path.join(path, "rgb")
    nb_path = os.path.join(path, "nb")
    os.makedirs(rgb_path, exist_ok=True)
    os.makedirs(nb_path, exist_ok=True)

    files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

    for file_name in tqdm(files, desc=f"[CLASSIFY] {category}", unit="image"):
        file_path = os.path.join(path, file_name)
        base_name, ext = os.path.splitext(file_name)

        try:
            img = Image.open(file_path).convert("RGB")
            np_img = np.array(img)

            # Vérifier si l'image est en noir et blanc ou couleur
            dest_folder = nb_path if is_grayscale(file_path) else rgb_path

            # Sauvegarde systématique en PNG
            new_filename = base_name + ".png"
            dest_path = os.path.join(dest_folder, new_filename)
            img.save(dest_path, "PNG")

            # Suppression des fichiers originaux non nécessaires
            if os.path.abspath(dest_path) != os.path.abspath(file_path):
                os.remove(file_path)

        except Exception as e:
            print(f"[ERROR] Fichier {file_path} : {e}")

In [ ]:
# =======================
# Partie 4 : Normalisation de la taille des images 
# =======================
for category in categories:
    path = os.path.join(working_dir, category)
    resized_path = os.path.join(path, "resized", "rgb")
    os.makedirs(resized_path, exist_ok=True)

    files = [f for f in os.listdir(os.path.join(path, "rgb")) if os.path.isfile(os.path.join(path, "rgb", f))]
    for file_name in tqdm(files, desc=f"[RESIZE] {category}", unit="image"):
        file_path = os.path.join(path, "rgb", file_name)
        base_name, ext = os.path.splitext(file_name)

        try:
            img = Image.open(file_path).convert("RGB")
            img_resized = img.resize((256, 256), Image.Resampling.LANCZOS)
            new_filename = base_name + ".png"
            dest_path = os.path.join(resized_path, new_filename)
            img_resized.save(dest_path, "PNG")

            # Suppression des fichiers originaux non nécessaires
            if os.path.abspath(dest_path) != os.path.abspath(file_path):
                os.remove(file_path)

        except Exception as e:
            print(f"[ERROR] Fichier {file_path} : {e}")
# Normalisation de la taille des images en noir et blanc
for category in categories:
    path = os.path.join(working_dir, category)
    resized_path = os.path.join(path, "resized", "nb")
    os.makedirs(resized_path, exist_ok=True)

    files = [f for f in os.listdir(os.path.join(path, "nb")) if os.path.isfile(os.path.join(path, "nb", f))]
    for file_name in tqdm(files, desc=f"[RESIZE] {category}", unit="image"):
        file_path = os.path.join(path, "nb", file_name)
        base_name, ext = os.path.splitext(file_name)

        try:
            img = Image.open(file_path).convert("RGB")
            img_resized = img.resize((256, 256), Image.Resampling.LANCZOS)
            new_filename = base_name + ".png"
            dest_path = os.path.join(resized_path, new_filename)
            img_resized.save(dest_path, "PNG")

            # Suppression des fichiers originaux non nécessaires
            if os.path.abspath(dest_path) != os.path.abspath(file_path):
                os.remove(file_path)

        except Exception as e:
            print(f"[ERROR] Fichier {file_path} : {e}")

# Suppression des dossiers temporaires
for category in categories:
    path = os.path.join(working_dir, category)
    shutil.rmtree(os.path.join(path, "rgb"), ignore_errors=True)
    shutil.rmtree(os.path.join(path, "nb"), ignore_errors=True)
# =======================
print("[INFO] Tri et redimensionnement des images terminés.")
# Display des images triées (nombre limité à 12)
def display_images_from_folder(folder_path):
    images = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.png')]
    images = images[:12]  # Limiter à 12 images
    plt.figure(figsize=(15, 10))
    for i, img_path in enumerate(images):
        img = Image.open(img_path)
        plt.subplot(3, 4, i + 1)
        plt.imshow(img)
        plt.axis('off')
    plt.show()
# Affichage des images triées
for category in categories:
    path = os.path.join(working_dir, category, "resized", "rgb")
    print(f"[DISPLAY] Images de la catégorie : {category}")
    display_images_from_folder(path)
# =======================
               


In [ ]:
# Paramètres
categories = ["Painting", "Photo", "Schematics", "Sketch", "Text"]
img_size = 256
batch_size = 32
epochs = 25
working_dir = "./images_work" 

###########################################################################
# 1. Préparation des données en classification binaire (Photo vs NonPhoto)  #
###########################################################################

def prepare_data_binary():
    """
    Organise les images issues des catégories existantes en deux classes :
    - "Photo" pour la catégorie Photo.
    - "NonPhoto" pour l’ensemble des autres catégories.
    
    Les images sont réparties dans des dossiers de train et de test (80%/20%).
    """
    binary_categories = ["Photo", "NonPhoto"]
    train_dir = os.path.join(working_dir, "train_binary")
    test_dir = os.path.join(working_dir, "test_binary")
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)
    
    # Création des dossiers de destination pour chaque classe binaire
    for cat in binary_categories:
        os.makedirs(os.path.join(train_dir, cat), exist_ok=True)
        os.makedirs(os.path.join(test_dir, cat), exist_ok=True)
    
    # Parcourir chaque catégorie d'origine
    for category in categories:
        # Détermine la classe binaire correspondante
        new_category = "Photo" if category == "Photo" else "NonPhoto"
        
        # Chemins des images en niveaux de gris (nb) et en RGB
        path_nb = os.path.join(working_dir, category, "resized", "nb")
        path_rgb = os.path.join(working_dir, category, "resized", "rgb")
        
        # Si les dossiers n'existent pas, on passe à la catégorie suivante
        if not os.path.exists(path_rgb) or not os.path.exists(path_nb):
            print(f"[WARNING] Les dossiers pour la catégorie {category} n'existent pas. Passage.")
            continue
        
        # Récupération des fichiers
        files_rgb = [f for f in os.listdir(path_rgb) if os.path.isfile(os.path.join(path_rgb, f))]
        files_nb = [f for f in os.listdir(path_nb) if os.path.isfile(os.path.join(path_nb, f))]
        
        # Division train / test (80%/20%)
        train_files_rgb, test_files_rgb = train_test_split(files_rgb, test_size=0.2, random_state=42)
        train_files_nb, test_files_nb = train_test_split(files_nb, test_size=0.2, random_state=42)
        
        # Copier les images RGB et NB dans les dossiers correspondants
        for f in tqdm(train_files_rgb, desc=f"[COPY] Train {category} RGB", unit="image"):
            shutil.copy(os.path.join(path_rgb, f), os.path.join(train_dir, new_category, f))
        for f in tqdm(test_files_rgb, desc=f"[COPY] Test {category} RGB", unit="image"):
            shutil.copy(os.path.join(path_rgb, f), os.path.join(test_dir, new_category, f))
        for f in tqdm(train_files_nb, desc=f"[COPY] Train {category} NB", unit="image"):
            shutil.copy(os.path.join(path_nb, f), os.path.join(train_dir, new_category, f))
        for f in tqdm(test_files_nb, desc=f"[COPY] Test {category} NB", unit="image"):
            shutil.copy(os.path.join(path_nb, f), os.path.join(test_dir, new_category, f))
    
    # Affichage des informations sur les dossiers créés
    print("[INFO] Structure des données binaires créée.")
    for cat in binary_categories:
        count_train = len(os.listdir(os.path.join(train_dir, cat)))
        count_test = len(os.listdir(os.path.join(test_dir, cat)))
        print(f"[INFO] {cat} - Train: {count_train}, Test: {count_test}")
    
    return train_dir, test_dir

###################################
# 2. Chargement des données       #
###################################

def load_data_binary(train_dir, test_dir):
    """
    Charge les données en appliquant une normalisation (rescale=1./255).
    Renvoie les générateurs pour l'entraînement et le test.
    """
    datagen = ImageDataGenerator(rescale=1./255)
    
    train_generator = datagen.flow_from_directory(
        train_dir,
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='binary',
        shuffle=True
    )
    
    test_generator = datagen.flow_from_directory(
        test_dir,
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='binary',
        shuffle=False
    )
    
    return train_generator, test_generator

###################################
# 3. Construction du modèle       #
###################################

def build_model_binary():
    """
    Construit un modèle CNN pour la classification binaire (Photo vs NonPhoto).
    """
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

###################################
# 4. Entraînement du modèle       #
###################################

def train_model_binary(model, train_generator, test_generator):
    """
    Entraîne le modèle en utilisant l'arrêt précoce et la sauvegarde du meilleur modèle.
    """
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint('best_model_binary.h5', save_best_only=True, monitor='val_loss')
    
    history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=test_generator,
        callbacks=[early_stopping, model_checkpoint]
    )
    
    return history

###################################
# 5. Évaluation du modèle         #
###################################

def evaluate_model_binary(model, test_generator):
    """
    Évalue le modèle sur les données de test, affiche le rapport de classification
    et trace la matrice de confusion.
    """
    test_loss, test_accuracy = model.evaluate(test_generator)
    print(f"[INFO] Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")
    
    y_pred = model.predict(test_generator)
    y_pred_class = (y_pred > 0.5).astype("int32")
    
    print("[INFO] Rapport de classification:")
    print(classification_report(test_generator.classes, y_pred_class))
    
    cm = confusion_matrix(test_generator.classes, y_pred_class)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=test_generator.class_indices.keys())
    disp.plot(cmap=plt.cm.Blues)
    plt.show()

###################################
# 6. Exemple de prédiction unique #
###################################

def detect_photo(model, image_path):
    """
    Charge une image, la prétraite, et effectue la prédiction.
    Affiche ensuite si l'image est une Photo ou non.
    """
    img = load_img(image_path, target_size=(img_size, img_size))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    
    pred = model.predict(img_array)
    if pred[0][0] > 0.5:
        print("L'image est une Photo.")
    else:
        print("L'image n'est pas une Photo.")

###################################
# 7. Fonction principale          #
###################################

def main():
    # Préparation et chargement des données en mode binaire
    train_dir, test_dir = prepare_data_binary()
    train_generator, test_generator = load_data_binary(train_dir, test_dir)
    
    # Construction et entraînement du modèle
    model = build_model_binary()
    history = train_model_binary(model, train_generator, test_generator)
    
    # Évaluation du modèle sur les données de test
    evaluate_model_binary(model, test_generator)
    
    # Exemple de détection sur une image individuelle (modifiez le chemin)
    image_path = "./dd/boat.png"
    detect_photo(model, image_path)

    # Affichage de l'historique d'entraînement
    plt.plot(history.history['accuracy'], label='train_accuracy')
    plt.plot(history.history['val_accuracy'], label='val_accuracy')
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    plt.show()
    # Affichage de la courbe de perte
    plt.plot(history.history['loss'], label='train_loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()   
    plt.show()

if __name__ == "__main__":
    main()